<a href="https://colab.research.google.com/github/pgordin/OptDisc2024/blob/main/Grafy4_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from random import random
from copy import deepcopy

In [2]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph
        self.weighted=False

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_array(cls, graph: np.array, nodes: list = None):
        if nodes is None:
            nodes = [*range(1, len(graph) + 1)]
        return cls.from_dict(
            cls._array_to_dict(graph, nodes)
        )

    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Generates a graphs from a text file, where each line defines one edge.\n
        Filename is a file path.
        """
        graph = Graph()
        file = open(filename, "r")
        for line in file:
            words = line.strip().split()
            if len(words) == 1:
                graph.add_node(words[0])
            elif len(words) > 1: # more than two words -- we use first two
                if directed:
                    graph.add_arc([words[0], words[1]])
                else:
                    graph.add_edge([words[0], words[1]])
        file.close()
        return graph

    @staticmethod
    def random_graph(nodes_num: int, prob: float):
        """
        Generates a random graph provided a number of nodes and probability of generating an edge.
        """
        rand_graph = Graph()
        for i in range(1, nodes_num + 1):
            rand_graph.add_node(i)
            for j in range(1, i):
                if random() < prob:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(nodes_num: int):
        """
        Generates a cycle provided a number of nodes.
        """
        cycle = Graph()
        nodes = [*range(1, nodes_num + 1)]
        for i in nodes:
            cycle.add_edge([nodes[i-2], nodes[i-1]])
        cycle.graph = dict(sorted(cycle.graph.items()))
        return cycle

    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")
        file.write(str(self))
        file.close()

    def nodes(self) -> list:
        """
        Returns list of nodes of a graph.
        """
        return [*self.graph.keys()]

    def array(self) -> np.array:
        """
        Returns the graph in array form.
        """
        return self._dict_to_array(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    def add_node(self, node):
        """
        Adds a node to a graph.
        """
        if node not in self.graph:
            self.graph[node] = []

    def del_node(self, node):
        """
        Recursively removes a node from a graph.
        """
        if node in self.graph:
            self.graph.pop(node)
            for key in [*self.graph.keys()]:
                if node in self.graph[key]:
                    self.graph[key].remove(node)

    def add_arc(self, arc: list):
        """
        Adds arc to a graph provided a list of nodes.
        """
        u, v = arc
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Adds edge to a graph provided a list of nodes.
        """
        u, v = edge
        if u == v:
            raise ValueError("Pętla!")
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    def _array_to_dict(arr: np.array, nodes: list) -> dict:
        """
        Converts a graph in array form to a graph in dict form.
        """
        res_dict = {}
        for i, node in enumerate(nodes):
            neighbours = [nodes[j] for j, edge in enumerate(arr[i]) if edge]
            res_dict[node] = neighbours
        return res_dict

    def _dict_to_array(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to a graph in array form.
        """
        n = len(_dict)
        nodes = [*_dict.keys()]
        res_arr = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (nodes.index(u), nodes.index(v))
            for u, row in _dict.items() for v in row
        ]:
            res_arr[u][v] += 1
        return res_arr

    def Prufer(self):
      """
      Kod Prufera drzewa - zwrócony jako napis
      Wymagane, aby graf był drzewem.
      Zwraca pusty napis dla drzew o mniej niż 3 wierzchołkach.
      """
      tr = deepcopy(self.graph)   # będziemy psuć graf
      code = ""
      for i in range(len(self.graph) - 2):
        for x in sorted(tr):    # po kolei przeglądam nieusunięte wierzchołki
          if len(tr[x]) == 1:   # najmniejszy liść
            break
        v = tr[x][0]  # sąsiad najmniejszego x
        code = code + f"{v} "
        tr[v].remove(x)   # usuwam x z listy sąsiadów v
        tr.pop(x)         # usuwam x z drzewa
      return code.strip()

    @staticmethod
    def tree_from_Prufer(code: str):
        """
        Tworzy drzewo na podstawie kodu Prufera.
        """
        tree = Graph()
        clist = [int(x) for x in code.strip().split()]   # kod zamieniamy na listę liczb
        n = len(clist) + 2    # liczba wierzchołków
        vert = [v for v in range(1, n+1)]  # lista liczb od 1 do n
        for v in vert:
          tree.add_node(v)
        for i in range(n-2):
          for x in vert:
            if not x in clist:    # najmniejszy liść
              break
          v = clist.pop(0)    # usuwam pierwszy element listy (sąsiad x)
          tree.add_edge((x, v))
          vert.remove(x)
        tree.add_edge(vert)
        return tree

    def ConnectedComponents(self):
      """
      Znajduje spójne składowe w grafie nieskierowanym
      Jako wynik zwraca listę zbiorów wierzchołków
      Uwaga: jako pierwszy element listy uzyskamy zbiór wszystkich wierzchołków grafu
      """
      def DFS(u):
        """
        Przeszukiwanie w głąb
        """
        for w in self.graph[u]:
          if not w in VT[0]:  # w jeszcze nie odwiedzony
            VT[0].add(w)      # już odwiedzony
            VT[-1].add(w)     # w ostatniej spójnej składowej
            DFS(w)
      """
      VT - lista zbiorów VT[i] dla i > 0 - lista wierzchołków spójnych składowych
      dla i = 0 - lista wszystkich odwiedzonych wierzchołków
      """
      VT = [set([])]
      for v in self.graph:
        if v not in VT[0]:
          VT[0].add(v)
          VT.append(set([v]))   # zaczątek nowej spójnej składowej
          DFS(v)
      return VT

    def Connected_components_graphs(self):
      """
      Spójne składowe jako grafy
      """
      VT=self.ConnectedComponents()
      noCc=len(VT)-1  #liczba spojnych skladowych
      graphs=[]  #lista do przechowywania grafow reprezentujacych spojne skladowe
      for i in range(noCc):  #iterujemy przez wszystkie spojne skladowe
          graph_i={}    #dla każdej spójnej składowej tworzymy nowy graf graph_i, który jest podgrafem oryginalnego grafu,
          for v in VT[i+1]:     #zawierającym tylko wierzchołki z bieżącej spójnej składowej
              graph_i[v]=self.graph[v].copy()
          graphs.append(Graph.from_dict(graph_i))  #dodajemy stworzone grafy do listy
      return graphs

    def Distance(self, v):
      """
      Znajduje i zwraca jako wektor słownik odległości od wierzchołka v
      do wierzchołków w tej samej spójnej składowej co v
      """
      dist = {v:0} # zalążek słownika odległości
      kolejka = [v]
      while len(kolejka) > 0:
        u = kolejka.pop(0)
        for w in self.graph[u]:
          if not w in dist:
            dist[w] = dist[u] + 1
            kolejka.append(w)
      return dist

    def GDFS(self, order=None):
        '''
        Uogólnione przechodzenie grafów skierowanych w głąb.
        Zwraca dwa słowniki (visited, processed)
        order - porządek wierzchołków (opcjonalny)
        '''
        
        def DFS(u):
            """
            wewnętrzna - przeszukiwanie w głąb
            """
            visited[u] = len(visited)+1
            for w in self.graph[u]:
                  if not w in visited:  # w jeszcze nie odwiedzony
                    DFS(w)
            processed[u]=len(visited)
            
        if order is None:
            vertices=self.graph.keys()
        else:
            vertices=order
        visited={}
        processed={}
        for v in vertices:
            if not v in visited:
                DFS(v)
        return visited, processed

# Przykłady wykorzystania

In [105]:
graph = Graph.random_graph(10, 1/5)
print(graph)
print("---------------------------")
print(graph.ConnectedComponents())


1:
2: 4
3:
4: 2
5: 9 10
6:
7:
8:
9: 5
10: 5

---------------------------
[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}, {1}, {2, 4}, {3}, {9, 10, 5}, {6}, {7}, {8}]


In [106]:
odl = graph.Distance(1)
print(odl)

{1: 0}


# Small world phenomenon - eksperyment Millgrama (1967)

Użyjemy grafu $G(n,p)$ (dokładnie $G(2000, 1/300)$) jako ilustrację zjawiska

In [3]:
import sys

n = 2000
p = 1/300
sys.setrecursionlimit(n+5)

rgraph = Graph.random_graph(n, p)
lista = rgraph.Connected_components_graphs()
graph = lista[0]
print(len(graph.graph))


2000


In [10]:
import time

start_time = time.time()
md = {}
ecc = {}
for v in graph.graph:
  dist = graph.Distance(v)
  ecc[v] = max(dist.values())
  md[v] = sum(dist.values())/len(dist.values())
print("Promień:", min(ecc.values()), " Średnica:", max(ecc.values()), " Średnio: ", sum(md.values())/len(md.values()))

end_time = time.time()
execution_time = end_time - start_time
print("Czas wykonania:", execution_time)

Promień: 6  Średnica: 8  Średnio:  4.2061085000000045
Czas wykonania: 2.9260246753692627


# Zadania
### Zadanie 1
Przy dużej liczbie wierzchołków przeszukiwanie w głąb natrafia na problem głębokości rekursji. Dorobić funkcję ConnectedComponentsBFS, która wykorzysta przeszukiwanie wszerz do wyznaczenia silnie spójnych składowych. Alternatywnie można spróbować napisać przeszukiwanie w głąb bez użycia wywołań rekursywnych

In [4]:
from collections import deque

def ConnectedComponentsBFS(graph):
    """
    Znajduje spójne składowe w grafie nieskierowanym jako wynik zwraca listę zbiorów wierzchołków
    """
    def BFS(v, visited, component):
        """
        Przeszukiwanie wszerz
        """
        queue = deque([v])     # dodajemy v do kolejki
        visited.add(v)         # oznaczamy v jako odwiedzony

        while queue:
            u = queue.popleft()    # oznaczamy pierwszy element z kolejki jako u
            component.add(u)       # dodajemy u do spójnej składowej
            for neighbor in graph.graph[u]:     # dla każdego sąsiada u
                if neighbor not in visited:     # jeśli sąsiad nie został odwiedzony to:
                    visited.add(neighbor)         # oznaczamy go jako odwiedzony
                    queue.append(neighbor)        # dodajemy go n koniec kolejki

    visited = set()   # na początku mamy pusty zbiór odwiedzonych wierzchołków
    components = []   # nie mamy jeszcze żadnych spójnych składowych

    for vert in graph.graph:                # dla każdego wierzchołka w grafie
        if vert not in visited:             # jeśli nie był on jeszcze odwiedzony
            component = set()                  # tworzymy nową spójną składową
            BFS(vert, visited, component)      # stosujemy dla tego wierzchołka procedurę BFS
            components.append(component)       # dodajemy spójną składową do wyniku
    return components


In [110]:
graph1 = Graph.random_graph(10, 1/5)
print(graph1)

1: 4 8 10
2: 6 7 8
3: 5 8
4: 1 5 7 9
5: 3 4
6: 2
7: 2 4
8: 1 2 3 9 10
9: 4 8
10: 1 8



In [111]:
ConnectedComponentsBFS(graph1)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}]

### Zadanie 2
Przedstawiony na zajęciach przykład (symulacja grafu małego świata można wykonać efektywniej z użyciem algorytmu Floyda-Warshala (do najkrótszych ścieżek). Sprawdzić na ile szybszy jest ten sposób jego realizacji.

In [5]:
def floyd_warshall(W):
    '''
    Algorytm Floyda-Warshalla.
    '''
    D = deepcopy(W)
    n = len(D)
    
    for k in range(n):
        for i in range(n):
            for j in range(n):
                D[i][j] = min(D[i][j], D[i][k] + D[k][j])

    return D

In [6]:
INF = float('inf')
def inf_matrix(rows, cols):
    """
    Funkcja tworząca macierz o wymiarach rows x cols, gdzie każdy element ma wartość INF.
    """
    return [[INF for _ in range(cols)] for _ in range(rows)]

In [7]:
def dict_to_dist_array(graph):
    '''
    Funkcja, która na podstawie grafu zwraca macierz odległości 
    (na i,j-tej pozycji jest liczba, która określa liczbę krawędzi, które
    trzeba przejść, aby z wierzchołka i przejść do wierzchołka j).
    
    Jeśli nie ma ścieżki łączącej wierzchołki i,j, na pozycji i,j-tej znajduje się Inf.
    '''
    n=len(graph.nodes())                                        # liczba wierzchołków
    W=inf_matrix(n,n)                                           # tworzymy macierz nxn z samych nieskończoności
    skladowe=ConnectedComponentsBFS(graph)                      # spójne składowe grafu
    for skladowa in skladowe:                                   # dla każdej spójnej składowej
        for skladowa_node in list(skladowa):                    # dla każdego wirzchołka w spójnej składowej
            dist=graph.Distance(skladowa_node)                 # obliczamy odległości
            keys=list(dist.keys())                              # zapisujemy dla jakiego wierzchołka
            values=list(dist.values())                          # jaka jest odległość
            for i in range(len(keys)):
                W[skladowa_node-1][keys[i]-1]=values[i]         # zapisujemy w macierzy
    return W

In [8]:
GRAPH = dict_to_dist_array(rgraph)
FW=floyd_warshall(GRAPH)

In [11]:
start_time = time.time()
md = []
ecc = []
for i in range(len(FW)):
    filtered_list = [x for x in FW[i] if x != INF]
    ecc.append(max(filtered_list))
    md.append(sum(filtered_list)/len(filtered_list))
print("Promień:", min(ecc), " Średnica:", max(ecc), " Średnio: ", sum(md)/len(md))

end_time = time.time()
execution_time_Floyd = end_time - start_time
print("Czas wykonania:", execution_time_Floyd)

Promień: 6  Średnica: 8  Średnio:  4.2061085000000045
Czas wykonania: 0.2903621196746826


### Zadanie 3
Napisać funkcję TopologicalSort która wykona sortowanie topologiczne grafu skierowanego (zadanie nieco trudniejsze, ale pseudokod był podany na wykładzie, a potrzebne elementy - na laboratorium).

In [ ]:
def TopologicalSort(graph):
    '''
    Funkcja wykonuje sortowanie topologiczne grafu skierowanego.
    '''
    vis, pro = graph.GDFS()                            # vis - visited, pro - processed
    
    n= len(graph.graph)                                # n - liczba wierzchołków
    order={}                                           # zaczątek słownika, który będzie określał porządek
    for v in graph.graph:
        order[v]=n*pro[v] - vis[v]                     # wyliczamy porządek
    porz = sorted(order, key=order.get, reverse=True)  # sortujemy słownik według porządku - malejąco
    return porz

In [11]:
ubranie_graph = Graph.from_edges("ubranie.txt", directed = 1)
print(ubranie_graph)

slipki: kalesony
kalesony: spodnie
spodnie: buty szelki
buty:
szelki: marynarka
skarpety: buty
koszula: szelki marynarka krawat
marynarka: plaszcz
krawat: marynarka
plaszcz:



In [12]:
TopologicalSort(ubranie_graph)

['koszula',
 'krawat',
 'skarpety',
 'slipki',
 'kalesony',
 'spodnie',
 'szelki',
 'marynarka',
 'plaszcz',
 'buty']

In [13]:
gra = {'a': ['b', 'e'],
       'b': ['c', 'd'],
       'c': ['a'],
       'd': ['e'],
       'e': ['d'],
       'f': ['e']}
Gra=Graph.from_dict(gra)
TopologicalSort(Gra)

['f', 'a', 'b', 'd', 'e', 'c']

### Zadanie 4
Dodać funkcję StronglyConnectedCoponents, która rozłoży graf skierowany na silnie spójne składowe. W wypadku grafu acyklicznego (dagu) będą one jednowierzchołkowe co da informację, że uzyskany porządek wierzchołków jest porządkiem topologicznym.

In [72]:
def StronglyConnectedCoponents(graph):
    """
    Rozkłada graf skierowany na silnie spójne składowe.
    """
    
    def transpose(graph):
        '''
        Funkcja, która zwraca graf przeciwny (czyli kierunki wszystkich łuków w grafie skierowanym są przeciwne do oryginalnego grafu).
        '''
        trans_graph = {}                           # tworzymy pusty słownik - zaczątek grafu przeciwnego
        for v in graph.graph:                      # dla każdego wierzchołka v w grafie
            trans_graph[v] = []                    # tworzymy jego odpowiednik w nowym grafie
            for u in graph.graph:                  # dla wierzchołka u w grafie
                if v in graph.graph[u]:            # jeśli v sąsiaduje z u
                    trans_graph[v] += [u]          # do wierzchołka v w przeciwnym grafie dodajemy jako sąsiada wierzchołek u
        return Graph.from_dict(trans_graph)

    porzadek = TopologicalSort(graph)              # wykonujemy sortowanie topologiczne i zapisujemy porządek
    graphT=transpose(graph)                        # tworzymy graf przeciwny do oryginalnego
    vis, pro = graphT.GDFS(porzadek)               # wyliczamy visited i processed dla grafu przeciwnego według porządku topologicznego oryginalnego grafu
    spojne_skladowe=[]                             # podwaliny silnie spójnych składowych
    
    # wyznaczamy spójne skadowe według schematu:
    # - szukamy najwcześniej odwiedzony wierzchołek
    # - sprawdzamy dla niego jaka jest wartość przetworzenia
    # - wszystkie wierzchołki, które mają nie większą wartość przetworzenia i nie mniejszą wartość odwiedzenia, klasyfikujemy jako wierzchołki w tej samej spójnej składowej
    # - ze zbioru wierzchołków wyrzucamy te, które już zaklasyfikowaliśmy do jednej spójnej składowej
    # - powtarzamy taki algorytm do sklasyfikowania wszystkich wirzchołków do którejś ze zspójnych składowych
    
    while vis:                                     # dopóki mamy wierzchołki bez przypisanej spójnej składowej
        wierzcholki=list(vis.keys())               # tworzymy listę 'niesklasyfikowanych' wierzchołków (od razu są w kolejności rosnącej według visited)
        v=wierzcholki[0]                           # wybieramy pierwszy wierzchołek (z najmniejszą wartośią visited)
        odwiedzony=vis[v]                          # zapamiętujemy wartość visited dla wybranego wierzchołka
        przetworzony=pro[v]                        # zapamiętujemy wartość processed dla wybranego wierzchołka
        skladowa=[v]                               # tworzymy spójną składową składającą się (póki co) z wybranego wierzchołka
        to_delete=[]                               # przygotowujemy listę, w której będziemy zapamiętywać wierzchołki, które zostały przypisane do tej spójnej składowej
        vis.pop(v)                                 # wyrzucamy ze zbioru wierzchołków bez przypisanej spójnej składowej wybrany wierzchołek
        for vertex in vis:                         # dla wszystkich pozostałych wierzchołków
            if pro[vertex]<=przetworzony:          # jeśli wartość przetworzenia jest nie większa niż wybranego wierzchołka
                if vis[vertex]>=odwiedzony:        # jeśli wartość odwiedzenia jest nie mniejsza niż wybranego wierzchołka
                    skladowa.append(vertex)        # dodajemy ten wierzchołek do spójnej składowej wybranego wierzchołka
                    to_delete.append(vertex)       # dodajemy ten wierzchołek do listy przetworzonych wierzchołków
        spojne_skladowe.append(skladowa)           # dodajemy stworzoną spójną składową do zbioru silnie spójnych składowych
        for i in to_delete:
            vis.pop(i)                             # usuwamy ze zbioru wierzchołków nieprzypisanych do spójnej składowej te, które zostały przetworzone
            
    return spojne_skladowe

        

In [60]:
ubranie_graph = Graph.from_edges("ubranie.txt", directed = 1)
print(ubranie_graph)

slipki: kalesony
kalesony: spodnie
spodnie: buty szelki
buty:
szelki: marynarka
skarpety: buty
koszula: szelki marynarka krawat
marynarka: plaszcz
krawat: marynarka
plaszcz:



In [73]:
print(StronglyConnectedCoponents(ubranie_graph))

[['koszula'], ['krawat'], ['skarpety'], ['slipki'], ['kalesony'], ['spodnie'], ['szelki'], ['marynarka'], ['plaszcz'], ['buty']]


In [74]:
gra = {'a': ['b', 'e'],
       'b': ['c', 'd'],
       'c': ['a'],
       'd': ['e'],
       'e': ['d'],
       'f': ['e']}
Gra=Graph.from_dict(gra)
StronglyConnectedCoponents(Gra)

[['f'], ['a', 'c', 'b'], ['d', 'e']]